In [1]:
# This is an example of the trsnafer learning or fine tuning of a model

In [4]:
# Importing libraries
import os
import numpy as np
import pandas as pd
import keras
import shutil
import tensorflow as tf
from keras import backend as K
from keras.layers import Conv2D, MaxPool2D, Dropout, Activation, Dense, Flatten, BatchNormalization
from keras.models import Sequential
from keras.layers.advanced_activations import LeakyReLU
from keras.preprocessing.image import ImageDataGenerator, img_to_array
from keras.preprocessing import image
from keras.metrics import categorical_crossentropy
from keras.layers.convolutional import *
from keras.applications import imagenet_utils
from keras.optimizers import adam
import matplotlib
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import itertools

In [5]:
files = os.listdir(os.getcwd() + '/Data/train')
files

['.DS_Store', '0', '1', '4', '3', '2', '5']

In [6]:
# Execute only once for splitting
'''
for i in ['0','1','2','3','4','5']:
    files = os.listdir(os.getcwd() + '/Data/train/' + i)
    leng = len(files)
    split_ratio1 = 0.15
    split_ratio2 = 0.14
    split_index1 = int(leng*split_ratio1)
    split_index2 = int(leng*(split_ratio1+split_ratio2))
    np.random.shuffle(files)
    for x in files[0:split_index1]:
        shutil.move(os.getcwd() + '/Data/train/' + i + '/' + x, os.getcwd() + '/Data/validation/' + i)
    for y in files[split_index1:split_index2]:
        shutil.move(os.getcwd() + '/Data/train/' + i + '/' + y, os.getcwd() + '/Data/test/' + i)
'''

"\nfor i in ['0','1','2','3','4','5']:\n    files = os.listdir(os.getcwd() + '/Data/train/' + i)\n    leng = len(files)\n    split_ratio1 = 0.15\n    split_ratio2 = 0.14\n    split_index1 = int(leng*split_ratio1)\n    split_index2 = int(leng*(split_ratio1+split_ratio2))\n    np.random.shuffle(files)\n    for x in files[0:split_index1]:\n        shutil.move(os.getcwd() + '/Data/train/' + i + '/' + x, os.getcwd() + '/Data/validation/' + i)\n    for y in files[split_index1:split_index2]:\n        shutil.move(os.getcwd() + '/Data/train/' + i + '/' + y, os.getcwd() + '/Data/test/' + i)\n"

In [7]:
# Loading the batches of train, test, and validation data
train_p = 'Data/train'
valid_p = 'Data/validation'
test_p = 'Data/test'

train_bches = ImageDataGenerator(preprocessing_function= keras.applications.mobilenet.preprocess_input) \
.flow_from_directory(train_p, target_size = (224,224), batch_size = 10)
valid_bches = ImageDataGenerator(preprocessing_function= keras.applications.mobilenet.preprocess_input) \
.flow_from_directory(train_p, target_size = (224,224), batch_size = 10)
test_bches = ImageDataGenerator(preprocessing_function= keras.applications.mobilenet.preprocess_input) \
.flow_from_directory(train_p, target_size = (224,224), batch_size = 10)

Found 885 images belonging to 6 classes.
Found 885 images belonging to 6 classes.
Found 885 images belonging to 6 classes.


In [8]:
def image_prep(x):
    img = image.load_img(x, target_size= (224,224))
    img_array = image.img_to_array(img)
    img = img_array.astype("float32")  
    img_dims = np.expand_dims(img, axis = 0)
    img = keras.applications.mobilenet.preprocess_input(img_dims)
    return img

In [9]:
model = keras.applications.mobilenet.MobileNet(include_top=False, weights= 'imagenet', input_shape= (224,224,3))

In [10]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (Activation)      (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128       
_________________________________________________________________
conv_dw_1_relu (Activation)  (None, 112, 112, 32)      0         
__________

In [11]:
# Defining model
def mobilenet():
    model = keras.applications.mobilenet.MobileNet(include_top=False, weights= 'imagenet', input_shape= (224,224,3))
    new_output = keras.layers.GlobalAveragePooling2D()(model.output)
    new_output = keras.layers.Dense(6, activation='softmax')(new_output)
    model = keras.engine.training.Model(model.inputs, new_output)
    return model

In [12]:
model = mobilenet()

In [13]:
len(model.layers)

84

In [14]:
img, labels = next(train_bches)
labels

array([[0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0., 0.]], dtype=float32)

In [15]:
for layer in model.layers[:-23]:
        layer.trainable = False

In [16]:
# Compiling model
model.compile(
    loss='categorical_crossentropy',  # we train 102-way classification
    optimizer=keras.optimizers.adamax(lr=1e-2),  # we can take big lr here because we fixed first layers
    metrics=['accuracy']  # report accuracy during training
)

In [17]:
model.fit_generator(train_bches, steps_per_epoch=15, validation_data=valid_bches, validation_steps= 3, epochs = 40,verbose=1)

Epoch 1/40
15/15 [==============================] - 16s 1s/step - loss: 1.9126 - acc: 0.6463 - val_loss: 5.2069 - val_acc: 0.6333
Epoch 2/40
15/15 [==============================] - 16s 1s/step - loss: 1.0704 - acc: 0.7267 - val_loss: 4.3449 - val_acc: 0.4667
Epoch 3/40
15/15 [==============================] - 16s 1s/step - loss: 0.4275 - acc: 0.8200 - val_loss: 2.5734 - val_acc: 0.4667
Epoch 4/40
15/15 [==============================] - 17s 1s/step - loss: 0.4664 - acc: 0.8933 - val_loss: 1.1128 - val_acc: 0.7000
Epoch 5/40
15/15 [==============================] - 17s 1s/step - loss: 0.1673 - acc: 0.9400 - val_loss: 0.3609 - val_acc: 0.8000
Epoch 6/40
15/15 [==============================] - 17s 1s/step - loss: 0.3116 - acc: 0.9133 - val_loss: 0.1008 - val_acc: 0.9667
Epoch 7/40
15/15 [==============================] - 16s 1s/step - loss: 0.1732 - acc: 0.9333 - val_loss: 0.9395 - val_acc: 0.7333
Epoch 8/40
15/15 [==============================] - 17s 1s/step - loss: 0.1090 - acc: 0.98

In [18]:
model.save('modeltransfer1.h5')
model.save_weights('modeltranferweights1.h5')

In [19]:
test_labels = test_bches.classes

In [20]:
predicts = model.predict_generator(test_bches, steps=5, verbose=1)
predicts

5/5 [==============================] - 4s 738ms/step


array([[9.40260421e-08, 2.77310563e-10, 8.00441849e-05, 9.68267727e-07,
        9.99642611e-01, 2.76282983e-04],
       [1.55250223e-06, 9.90899920e-01, 9.06031299e-03, 3.62302817e-05,
        2.02118076e-06, 8.26575299e-13],
       [2.79972621e-04, 9.98494148e-01, 5.35922998e-04, 2.14446583e-04,
        4.74993256e-04, 4.44793642e-07],
       [1.16544516e-06, 2.56364201e-05, 1.80969437e-04, 9.99748647e-01,
        7.05032164e-07, 4.29003449e-05],
       [4.14905969e-07, 1.64542442e-08, 2.50527170e-04, 4.00189123e-07,
        9.99509335e-01, 2.39260888e-04],
       [2.89640838e-07, 1.72361160e-05, 2.85015581e-03, 9.97132063e-01,
        1.29001478e-07, 1.21003097e-07],
       [5.51167489e-07, 2.58674231e-06, 2.89230462e-04, 9.99682546e-01,
        8.00480962e-08, 2.50561607e-05],
       [9.99881864e-01, 3.74942429e-05, 6.95908602e-06, 6.82504178e-05,
        5.14911244e-06, 1.81860301e-07],
       [7.23455798e-07, 9.97506917e-01, 2.48450926e-03, 7.27129054e-06,
        5.76663865e-07, 

In [161]:
predicts.argmax(axis=1)

array([1, 0, 1, 0, 0, 1, 2, 4, 1, 1, 5, 0, 5, 0, 1, 2, 1, 4, 5, 5, 2, 0,
       3, 1, 3, 4, 3, 2, 0, 4, 1, 4, 2, 1, 0, 2, 1, 3, 5, 0, 3, 3, 4, 0,
       0, 2, 2, 1, 0, 4])

In [162]:
#cm = confusion_matrix(test_labels, predicts.argmax(axis=1))

ValueError: Found input variables with inconsistent numbers of samples: [885, 50]

In [174]:
test_im, test_labels = next(test_bches)

In [175]:
model.predict(test_im).argmax(axis=1)

array([4, 4, 4, 5, 4, 3, 4, 0, 5, 4])

In [176]:
test_labels.argmax(axis=1)

array([4, 4, 4, 5, 4, 3, 4, 0, 5, 4])

In [2]:
from keras.models import load_model
model = load_model("modeltransfer.h5")

ValueError: Unknown activation function:relu6

In [3]:
from keras.utils.generic_utils import CustomObjectScope

with CustomObjectScope({'relu6': keras.applications.mobilenet.relu6,'DepthwiseConv2D': keras.applications.mobilenet.DepthwiseConv2D}):
    model = load_model('modeltransfer.h5')

In [ ]:
model.predict("")